# DSC650 Assignment 10.4 Ryan Gough

In [1]:
from pathlib import Path
import os
import random
import sys

In [9]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

2022-08-12 20:41:51.682971: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


###### Assignment 11

Using section 8.1 in Deep Learning with Python as a guide, implement an LSTM text generator. 

Train the model on the Enron corpus or a text source of your choice. 

Save the model and generate 20 examples to the results directory of dsc650/assignments/assignment11/

In [10]:
# Establish Working and Results Directory
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('Documents/GitHub/dsc650/dsc650/assignments/assignment11')
results_dir.mkdir(parents=True, exist_ok=True)

In [11]:
# Open Dracula text.
dracula = open("/Users/ryangough/Downloads/345-0.txt").read().lower()

In [12]:
# create mapping of unique chars to integers
chars = sorted(list(set(dracula)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [13]:
# Get text info. 
n_chars = len(dracula)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  865289
Total Vocab:  78


In [14]:
# prepare the dataset of input to output pairs encoded as integers.

seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = dracula[i:i + seq_length]
    seq_out = dracula[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_patterns = len(dataX)

print ("Total Patterns: ", n_patterns)

Total Patterns:  865189


In [15]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [16]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

2022-08-12 20:42:17.231135: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [18]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [19]:
# For 4 Temperatures, use a for-loop 5 times to generate 20 results. Printing results instead saving to a text.

maxlen=100
for epoch in range(1, 5):
    print('Epoch', epoch)
    # Fit the model for 1 epoch on the data
    model.fit(X, y, batch_size=128, epochs=1, callbacks=callbacks_list)

    # Select a text seed at random
    start_index = random.randint(0, len(dracula) - maxlen - 1)
    generated_text = dracula[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Epoch 1
 207/6760 [..............................] - ETA: 17:31 - loss: 3.0910

KeyboardInterrupt: 